In [5]:
import numpy as np
import copy
from enum import Enum

In [4]:
# 式（5）
def get_Q_out_H_hs(_equipment, _Q_dmd_H_hs):
    
    if _equipment == 'gasConv' or 'gasCondensing':
        _Q_max_H_hs = 1 # 暫定　付録Bの最大暖房出力を呼び出す
    
    
    
    return min(_Q_dmd_H_hs , _Q_max_H_hs)

In [6]:
get_Q_out_H_hs(_equipment = 'gasConv', _Q_dmd_H_hs = 10)

1

In [2]:
# 式（7）　温水供給運転率
def get_r_WS_hs(_Q_dmd_H_hs, _r_WS_ln_i):
    if _Q_dmd_H_hs > 0:
        
        # 放熱系統の数を温水供給運転率の値の数で判定
        rad_num = 0
        for i in _r_WS_ln_i:
            if type(i) == int or type(i) == float:
                rad_num += 1
                _r_WS_hs = i

        if rad_num == 1:
            return _r_WS_hs
        elif rad_num >= 2:
            return 1
    elif _Q_dmd_H_hs == 0:
        return 0

#### Example

In [4]:
get_r_WS_hs(_Q_dmd_H_hs=1, _r_WS_ln_i=[None, None, 0.2, None, None])

0.2

In [8]:
# 式(9)
def get_theta_RW_hs(_theta_SW_hs, _Q_dmd_H_ln_i, _Q_max_H_rad_i):
    
    i = 0
    _Q_dmd_H_ln_i = np.array(_Q_dmd_H_ln_i)
    _Q_dash_max_H_rad_i = copy.deepcopy(_Q_dmd_H_ln_i)
    
    for d in _Q_dmd_H_ln_i:
        if d > 0:
            _Q_dash_max_H_rad_i[i] = _Q_max_H_rad_i[i]
        elif d == 0:
            _Q_dash_max_H_rad_i[i] = 0
        i += 1
    print('_Q_dash_max_H_rad_i =', _Q_dash_max_H_rad_i, '_Q_dmd_H_ln_i =', _Q_dmd_H_ln_i)
    
    _T_dif = np.sum(_Q_dmd_H_ln_i) / (np.sum(_Q_dash_max_H_rad_i) * 0.027583)
    print(_T_dif)    
    
    if 40 < _theta_SW_hs <= 60:
        return 0.0301 * _T_dif**2 - 0.1864 * _T_dif + 20
    elif _theta_SW_hs <= 40:
        return 0.0604 * _T_dif**2 - 0.1881 * _T_dif + 20
    else:
        raise ValueError("_theta_SW_hsの値が不正です。")


#### Example

In [6]:
a = [100, 0, 500, 300, 200]
print(type(a))

b = np.array(a)
print(type(b))

<class 'list'>
<class 'numpy.ndarray'>


In [9]:
get_theta_RW_hs(_theta_SW_hs = 60, _Q_dmd_H_ln_i = [100, 0, 500, 300, 200], _Q_max_H_rad_i = [20, 15, 10, 5, 4])

_Q_dash_max_H_rad_i = [20  0 10  5  4] _Q_dmd_H_ln_i = [100   0 500 300 200]
1022.55476943


31302.505311015513

In [10]:
# 式(10)
get_Q_dmd_H_ln_i = lambda _Q_dmd_H_rad_i, _Q_loss_pp_i : [x + y for (x, y) in zip(_Q_dmd_H_rad_i, _Q_loss_pp_i)]


In [11]:
# 式(10)
def get_Q_dmd_H_ln_i2(_Q_dmd_H_rad_i, _Q_loss_pp_i):
    _Q_dmd_H_rad_i = np.array(_Q_dmd_H_rad_i)
    _Q_loss_pp_i   = np.array(_Q_loss_pp_i)
    
    return _Q_dmd_H_rad_i + _Q_loss_pp_i

#### Example

In [13]:
get_Q_dmd_H_ln_i2(_Q_dmd_H_rad_i = [1,2,3,4,5], _Q_loss_pp_i = [10,20,30,40,50])

array([11, 22, 33, 44, 55])

In [14]:
# 式(11)
def get_r_WS_ln_i(_r_WS_rad_i):
    return np.array(_r_WS_rad_i)

#### Example

In [15]:
get_r_WS_ln_i(_r_WS_rad_i = [0.4, 0, 0,1, 0.8, 1])

array([ 0.4,  0. ,  0. ,  1. ,  0.8,  1. ])

In [19]:
class Pipe_Insulation(Enum):
    Insulation = 1,
    NotInsulation = 2

In [18]:
class Under_Floor_Pipe(Enum):
    All_Insulation = 1,
    Some_Insulation = 2

In [185]:
{'Pipe_Insulation.Insulation' : 1, 'Pipe_Insulation.NotInsulation' : 3}['Pipe_Insulation.NotInsulation']

3

In [20]:
# 式(12)
def get_Q_loss_pp_i(_theta_SW, _theta_ex, _A_A, _under_floor_pipe, _pipe_insulation, _r_WS_rad_i):
    
    _K_loss_pp_i = {Pipe_Insulation.Insulation : (0.21, 0.21, 0.21, 0.21, 0.21), Pipe_Insulation.NotInsulation : (0.15, 0.15, 0.15, 0.15, 0.15)}[_pipe_insulation]
    
    _L_pp_R_1 = 120
    
    _L_pp_ex_i_R, _L_pp_in_i_R = {Under_Floor_Pipe.Some_Insulation : {'_L_pp_ex_R' : (0.00     , 0.00, 0.00, 0.00), '_L_pp_in_R' : (_L_pp_R_1, 22.86, 19.22, 26.62)},
                                  Under_Floor_Pipe.All_Insulation  : {'_L_pp_ex_R' : (_L_pp_R_1, 0.00, 0.00, 0.00), '_L_pp_in_R' : (0.00     , 22.86, 19.22, 26.62)}}[_under_floor_pipe]
    
    print(_L_pp_ex_i_R, _L_pp_in_i_R)
    
    _A_A_R = 120.08
    if _under_floor_pipe == Under_Floor_Pipe.Some_Insulation:
        _L_pp_ex_i = _L_pp_ex_i_R * np.sqrt(_A_A / _A_A_R)
    elif _under_floor_pipe == Under_Floor_Pipe.All_Insulation:
        _L_pp_in_i = _L_pp_in_i_R * np.sqrt(_A_A / _A_A_R)


In [192]:
aa, bb = {'a' : (1,2), 'b':(3,4)}['b']
aa, bb

(3, 4)

In [21]:
get_Q_loss_pp_i(_theta_SW = 20, _theta_ex = 40, _A_A=60, _under_floor_pipe = Under_Floor_Pipe.All_Insulation, _pipe_insulation = Pipe_Insulation.Insulation, _r_WS_rad_i = [10,10,10,10,10])

_L_pp_ex_R _L_pp_in_R


TypeError: 'numpy.float64' object cannot be interpreted as an integer